In [1]:
# 필수 라이브러리 설치
#!pip install -q -U transformers accelerate bitsandbytes datasets rouge-score

In [2]:
# 구글 드라이브 마운트
from google.colab import drive
import os
import torch

drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [3]:
# 데이터 및 프로젝트 경로 설정
BASE_DIR = "/content/drive/MyDrive/Buffalo_Wing"
DATA_DIR = os.path.join(BASE_DIR, "train")

# 데이터 파일 경로
DATA_PATHS = {
    "sft_data": os.path.join(DATA_DIR, "SFTdata.json"),
    "sft_label": os.path.join(DATA_DIR, "SFTlabel.json"),
    "rm_data": os.path.join(DATA_DIR, "RMdata.json"),
    "rm_label": os.path.join(DATA_DIR, "RMlabel.json"),
    "ppo_data": os.path.join(DATA_DIR, "PPOdata.json"),
}

In [4]:
import json
import random

# 모델 리스트
MODELS = {
    "solar": "upstage/SOLAR-10.7B-v1.0",           # 상위권 (한국어 특화)
    # "llama3": "meta-llama/Meta-Llama-3-8B",        # 글로벌 표준
    # "qwen": "Qwen/Qwen2-7B",                       # 아시아어 강자
    # "polyglot": "EleutherAI/polyglot-ko-1.3b",     # 멍청한 모델 1 (비교용)
    # "kogpt2": "skt/kogpt2-base-v2",                 # 멍청한 모델 2 (하한선)
    "kanana": "kakaocorp/kanana-nano-2.1b-instruct",   # 카카오 최신 모델 (추가)
    "exaone": "LGAI-EXAONE/EXAONE-4.0-1.2B", # LG 최신 경량 모델 (추가)
    "kanana2": "kakaocorp/kanana-1.5-8b-instruct-2505"
}

# 3. 샘플 테스트 및 추론 설정 [cite: 11]
INFERENCE_CONFIG = {
    "sample_size": 10,          # [수정] 빠른 테스트를 위해 100개 -> 10개로 축소했습니다.
    "max_new_tokens": 128,      # 생성할 답변의 최대 길이
    "temperature": 0.7,         # 창의성 조절 (0에 가까울수록 결정론적)
    "top_p": 0.9,               # 답변의 다양성 조절
    "repetition_penalty": 1.1   # 동일 문구 반복 방지
}

In [5]:
# 4. 결과 저장 경로
RESULT_DIR = os.path.join(BASE_DIR, "results")
if not os.path.exists(RESULT_DIR):
    os.makedirs(RESULT_DIR)

In [6]:
from huggingface_hub import notebook_login
notebook_login() # 실행 후 토큰 입력

In [7]:
def load_and_prepare_data():
    print("🔄 데이터 로딩 및 정밀 병합 시작...")

    try:
        with open(DATA_PATHS['sft_data'], 'r', encoding='utf-8') as f:
            sft_raw = json.load(f)
        with open(DATA_PATHS['sft_label'], 'r', encoding='utf-8') as f:
            label_raw = json.load(f)

        # data_info 섹션 추출
        sft_items = sft_raw.get('data_info', [])
        label_items = label_raw.get('data_info', [])
    except Exception as e:
        print(f"❌ 파일 읽기 에러: {e}")
        return None

    # 1. 라벨 데이터를 data_id 기준으로 딕셔너리화
    label_map = {str(item.get('data_id')): item for item in label_items if isinstance(item, dict)}

    merged_list = []

    # 2. 질문 데이터와 매칭
    for item in sft_items:
        d_id = str(item.get('data_id'))

        # 질문 내용 추출 (분석된 'question' 키 사용)
        prompt = item.get('question')

        if d_id in label_map:
            label_item = label_map[d_id]
            # 정답 내용 추출 (분석된 answer -> contents 구조 반영)
            answer_dict = label_item.get('answer', {})
            reference = answer_dict.get('contents') if isinstance(answer_dict, dict) else None

            # 카테고리 추출 (중첩된 구조 처리)
            category_info = item.get('data_category', {})
            category = category_info.get('middle') if isinstance(category_info, dict) else '일반'

            if prompt and reference:
                merged_list.append({
                    "idx": d_id,
                    "prompt": prompt,
                    "reference": reference,
                    "category": category
                })

    print(f"✅ 병합 완료: 총 {len(merged_list)}개의 유효 데이터를 확보했습니다.")
    return merged_list

# 7번 셀 실행
merged_data = load_and_prepare_data()

# 데이터 검증 출력
if merged_data and len(merged_data) > 0:
    print("\n🔍 병합 데이터 검증 (첫 번째 항목):")
    print(f"ID: {merged_data[0]['idx']}")
    print(f"질문: {merged_data[0]['prompt'][:50]}...")
    print(f"정답: {merged_data[0]['reference'][:50]}...")

    # 설정한 샘플 사이즈만큼 무작위 추출
    sample_size = min(INFERENCE_CONFIG['sample_size'], len(merged_data))
    test_questions = random.sample(merged_data, sample_size)
    print(f"\n🎯 실험용 샘플 {len(test_questions)}개가 준비되었습니다.")

🔄 데이터 로딩 및 정밀 병합 시작...
✅ 병합 완료: 총 10580개의 유효 데이터를 확보했습니다.

🔍 병합 데이터 검증 (첫 번째 항목):
ID: 04779be1-67ac-4b70-8dc2-fda6dc353b2a
질문: 화학공학이 다른 공학 분야와 어떻게 다른가요?...
정답: 화학공학은 화학과 공학의 합성으로 이루어진 학문 분야입니다. 다른 일반적인 공학 분야와 비...

🎯 실험용 샘플 10개가 준비되었습니다.


In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import time
import json
import os

all_model_results = {}

for model_name, model_path in MODELS.items():
    print(f"\n" + "="*50)
    print(f"🚀 테스트 중: {model_name.upper()}")
    print(f"경로: {model_path}")
    print("="*50)

    try:
        # 1. 토크나이저 및 모델 로드
        tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

        # 패딩 토큰 강제 설정
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token

        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            torch_dtype=torch.bfloat16,
            device_map={"": 0}, # A100 40GB 전용 할당
            trust_remote_code=True
        )
        model.eval()

        current_outputs = []
        start_time = time.time()

        for i, sample in enumerate(test_questions):
            prompt = sample.get('prompt', "")

            # 2. Chat Template 적용 (EXAONE, KANANA 지시 이행 필수)
            # 모델이 '지시'임을 인지하게 하여 빈 답변 방지
            messages = [{"role": "user", "content": prompt}]
            try:
                # 템플릿 적용 시도
                input_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            except:
                # 템플릿 미지원 시 일반 프롬프트 사용
                input_text = prompt

            inputs = tokenizer(input_text, return_tensors="pt", add_special_tokens=True).to(0)

            # Polyglot 에러 방지 (Unused kwargs 제거)
            if 'token_type_ids' in inputs:
                del inputs['token_type_ids']

            # 3. 답변 생성
            with torch.no_grad():
                output_tokens = model.generate(
                    **inputs,
                    max_new_tokens=INFERENCE_CONFIG['max_new_tokens'],
                    temperature=0.1, # 안정적인 답변을 위해 낮춤
                    top_p=0.9,
                    repetition_penalty=1.2,
                    do_sample=True,
                    pad_token_id=tokenizer.pad_token_id,
                    eos_token_id=tokenizer.eos_token_id
                )

            # 생성된 부분만 디코딩
            generated_text = tokenizer.decode(output_tokens[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)

            current_outputs.append({
                "idx": sample['idx'],
                "prompt": prompt,
                "reference": sample.get('reference', ''),
                "model_output": generated_text.strip()
            })

            if (i + 1) % 2 == 0:
                print(f"   - 진행 상황: {i + 1}/{len(test_questions)} 완료")

        all_model_results[model_name] = current_outputs
        print(f"✅ {model_name} 완료! (소요 시간: {int(time.time() - start_time)}초)")

    except Exception as e:
        print(f"❌ {model_name} 에러: {e}")

    finally:
        # 4. 메모리 완전 소거 (A100 VRAM 40GB 확보용)
        if 'model' in locals(): del model
        if 'tokenizer' in locals(): del tokenizer
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
        print(f"🧹 {model_name} 메모리 정리 완료")

# 결과 저장
output_file = os.path.join(RESULT_DIR, "baseline_inference_results.json")
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(all_model_results, f, ensure_ascii=False, indent=4)


🚀 테스트 중: SOLAR
경로: upstage/SOLAR-10.7B-v1.0


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/966 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

...

model-00001-of-00004.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

   - 진행 상황: 2/10 완료
   - 진행 상황: 4/10 완료
   - 진행 상황: 6/10 완료
   - 진행 상황: 8/10 완료
   - 진행 상황: 10/10 완료
✅ kanana2 완료! (소요 시간: 47초)
🧹 kanana2 메모리 정리 완료


In [9]:
# rouge-score 라이브러리 설치
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=14a3095bf85c8835413e00b1184f626924241eb001e0b6188093d0fefd1d79b4
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge-score


In [12]:
import pandas as pd
from rouge_score import rouge_scorer

def calculate_baseline_metrics():
    print("📊 정량적 분석 시작 (ROUGE Score 계산)...")

    result_path = os.path.join(RESULT_DIR, "baseline_inference_results.json")
    if not os.path.exists(result_path):
        print("❌ 결과 파일을 찾을 수 없습니다. 8번 셀이 성공적으로 끝났는지 확인하세요.")
        return

    with open(result_path, 'r', encoding='utf-8') as f:
        results = json.load(f)

    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    summary_data = []

    for model_name, outputs in results.items():
        if not outputs: continue
        total_r1, total_rl, total_len = 0, 0, 0
        count = len(outputs)

        for item in outputs:
            ref = item['reference']
            gen = item['model_output']
            scores = scorer.score(ref, gen)
            total_r1 += scores['rouge1'].fmeasure
            total_rl += scores['rougeL'].fmeasure
            total_len += len(gen)

        summary_data.append({
            "Model": model_name.upper(),
            "ROUGE-1": round(total_r1 / count, 4),
            "ROUGE-L": round(total_rl / count, 4),
            "Avg_Length": round(total_len / count, 1)
        })

    df = pd.DataFrame(summary_data)
    print("\n[ 베이스라인 모델 성능 비교 ]")
    display(df.sort_values(by="ROUGE-L", ascending=False))

    df.to_csv(os.path.join(RESULT_DIR, "baseline_metrics_summary.csv"), index=False)

calculate_baseline_metrics()

📊 정량적 분석 시작 (ROUGE Score 계산)...

[ 베이스라인 모델 성능 비교 ]


,Model,ROUGE-1,ROUGE-L,Avg_Length
2,EXAONE,0.2494,0.2494,266.8
1,KANANA,0.2299,0.2299,240.4
3,KANANA2,0.1446,0.1446,230.0
0,SOLAR,0.1033,0.0964,286.3


In [11]:
from IPython.display import HTML, display as ipy_display

def generate_comparison_report():
    result_path = os.path.join(RESULT_DIR, "baseline_inference_results.json")
    with open(result_path, 'r', encoding='utf-8') as f:
        results = json.load(f)

    html_content = """
    <style>
        .report-table { width: 100%; border-collapse: collapse; font-family: sans-serif; }
        .report-table th, .report-table td { border: 1px solid #ddd; padding: 10px; }
        .report-table th { background-color: #f4f4f4; }
        .prompt-row { background-color: #e7f3ff; font-weight: bold; }
        .ref-row { background-color: #f9f9f9; color: #666; font-style: italic; }
    </style>
    <h2>📝 모델별 답변 정성적 비교 리포트</h2>
    <table class='report-table'>
    """

    model_names = list(results.keys())
    if not model_names: return
    num_samples = len(results[model_names[0]])

    for i in range(num_samples):
        sample = results[model_names[0]][i]
        html_content += f"<tr class='prompt-row'><td colspan='2'>질문 {i+1}: {sample['prompt']}</td></tr>"
        html_content += f"<tr class='ref-row'><td colspan='2'>정답: {sample['reference']}</td></tr>"

        for name in model_names:
            if i < len(results[name]):
                output = results[name][i]['model_output']
                html_content += f"<tr><td style='width:120px; font-weight:bold;'>{name.upper()}</td><td>{output}</td></tr>"

    html_content += "</table>"
    ipy_display(HTML(html_content))

generate_comparison_report()